In [1]:
# === P-HSUMI — Percentiles de la MÁXIMA suma 30d por cliente (Inbound Cash) ===
# LÓGICA EXACTA (parametrización):
#   tx_direction = Inbound AND tx_type = Cash
#   Por cliente: S30(t) = suma base en los últimos 30 días (incluye t)
#   Tomamos max_{t}(S30) por cliente y calculamos percentiles sobre esos máximos.

import pandas as pd, numpy as np

pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# -------- EDITA AQUÍ -----------------------------------------------------------
PATH = "../../data/tx/datos_trx__with_subsub.csv"
SUBSUBSEGMENTS = "R-Low"     # <-- ajusta el sub-subsegmento
PCTS = [0.85, 0.90, 0.95, 0.97, 0.99]     # percentiles a reportar
# ------------------------------------------------------------------------------

# Carga y limpieza mínima
df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df.get("tx_direction","").astype(str).str.title()
df["tx_type"]        = df.get("tx_type","").astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

# Filtro: IN + Cash + datos válidos
g = df[
    df["tx_direction"].eq("Inbound")
    & df["tx_type"].eq("Cash")
    & df["tx_date_time"].notna()
    & df["tx_base_amount"].notna()
][["customer_id","tx_date_time","tx_base_amount"]].copy()

if g.empty:
    print("P-HSUMI: No hay transacciones elegibles.")
else:
    max_rows = []
    for cid, sub in g.groupby("customer_id", sort=False):
        daily = (sub.set_index("tx_date_time")["tx_base_amount"]
                   .abs()
                   .resample("D").sum())
        if daily.empty:
            continue
        S30 = daily.rolling("30D").sum()
        max_rows.append({"customer_id": cid, "S30_max": float(S30.max())})

    R = pd.DataFrame(max_rows)
    if R.empty:
        print("P-HSUMI: No se pudieron construir ventanas 30d.")
    else:
        s = R["S30_max"].astype(float)
        q = s.quantile(PCTS) if len(s) else pd.Series(index=PCTS, dtype=float)

        out = pd.DataFrame({
            "percentil": [f"p{int(p*100)}" for p in PCTS],
            "Amount_30d_max_per_customer_CLP": [q.get(p, np.nan) for p in PCTS]
        })

        print("=== P-HSUMI — Máxima suma CLP en 30 días por cliente (percentiles) ===")
        print(f"Clientes considerados: {s.shape[0]:,}")
        display(out)

        if pd.notna(q.get(0.95, np.nan)):
            print(f"\nSugerencia {{var.Amount}} (p95 máx-30d por cliente): {q.get(0.95):,.0f} CLP")


C:\Users\vinso\AppData\Local\Temp\ipykernel_20164\3546171157.py:18: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-HSUMI — Máxima suma CLP en 30 días por cliente (percentiles) ===
Clientes considerados: 8,065


,percentil,Amount_30d_max_per_customer_CLP
0,p85,"89,078,274"
1,p90,"125,080,290"
2,p95,"220,000,000"
3,p97,"295,076,065"
4,p99,"541,544,706"



Sugerencia {var.Amount} (p95 máx-30d por cliente): 220,000,000 CLP


# Simulación alertas

In [5]:
# === P-HSUMI — Sensibilidad (Actual vs propuestos) =============================
# Lógica: tx_direction=Inbound & tx_type=Cash; SUM_30d > Amount
# Unidad = ventanas cliente–día

import pandas as pd, numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

PATH = "../../data/tx/transacciones_cash_2025__with_subsub.csv"
SUBSUBSEGMENTS = ["I-2"]                # <-- ajusta el sub-subsegmento
PARAMS = {
    "Actual": {"Amount": 9_662_835_000},
   # "p85":    {"Amount":   402_334_913},
    #"p90":    {"Amount":   603_000_000},
    "p95":    {"Amount":16_861_936_597},
    "p97":    {"Amount": 17_640_863_445},
    "p99":    {"Amount": 18_419_790_292},
}

df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")
df["tx_date_time"]   = pd.to_datetime(df["tx_date_time"], errors="coerce")
df["tx_base_amount"] = pd.to_numeric(df["tx_base_amount"], errors="coerce")
df["tx_direction"]   = df["tx_direction"].astype(str).str.title()
df["tx_type"]        = df["tx_type"].astype(str).str.title()

# Filtrado por sub-subsegmento
if isinstance(SUBSUBSEGMENTS, str):
    target_labels = {SUBSUBSEGMENTS}
else:
    target_labels = set(map(str, SUBSUBSEGMENTS))

df = df[df["customer_sub_sub_type"].astype(str).isin(target_labels)].copy()

IN_ = df[(df["tx_direction"].eq("Inbound")) & (df["tx_type"].eq("Cash")) & df["tx_base_amount"].notna() & df["tx_date_time"].notna()]
parts=[]
for cid, sub in IN_.groupby("customer_id", sort=False):
    daily = sub.set_index("tx_date_time")["tx_base_amount"].abs().resample("D").sum()
    S30  = daily.rolling("30D").sum()
    parts.append(pd.DataFrame({"customer_id": cid, "date": S30.index, "S30": S30.values}))
M = pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(columns=["customer_id","date","S30"])

order = ["Actual","p85","p90","p95","p97","p99"]
param_tbl = pd.DataFrame(PARAMS).T.loc[[k for k in order if k in PARAMS]].rename_axis("escenario").reset_index()
print("=== P-HSUMI — Parámetros (Amount) ==="); display(param_tbl)

counts = {k: int(M.loc[M["S30"] > v["Amount"], ["customer_id","date"]].drop_duplicates().shape[0]) for k,v in PARAMS.items()}

out = pd.DataFrame([{
    "alertas_actual": counts.get("Actual", 0),
    "alertas_p85":    counts.get("p85", 0),
    "alertas_p90":    counts.get("p90", 0),
    "alertas_p95":    counts.get("p95", 0),
    "alertas_p97":    counts.get("p97", 0),
    "alertas_p99":    counts.get("p99", 0),
}])
print("=== P-HSUMI — Alertas por escenario (ventanas cliente–día) ==="); display(out)


C:\Users\vinso\AppData\Local\Temp\ipykernel_26472\1463855063.py:19: DtypeWarning: Columns (70,71) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(PATH, dtype={"customer_id":"string"}, encoding="utf-8-sig")


=== P-HSUMI — Parámetros (Amount) ===


,escenario,Amount
0,Actual,9662835000
1,p95,16861936597
2,p97,17640863445
3,p99,18419790292


=== P-HSUMI — Alertas por escenario (ventanas cliente–día) ===


,alertas_actual,alertas_p85,alertas_p90,alertas_p95,alertas_p97,alertas_p99
0,244,0,0,1,1,1


In [9]:
# === P-HSUMI — Simulación (gatillo por transacción) vs Reales ==================
# Regla: tx_direction = Inbound [AND tx_type="Cash"] ; S30_after(tx) > Amount
# Gatillo: por transacción que sobrepasa el umbral (S30_before <= A < S30_after)
# Historial: usa todo; conteo/matching: desde COUNT_FROM (y opcional COUNT_TO)

import pandas as pd
import numpy as np
pd.set_option("display.float_format", lambda x: f"{x:,.0f}")

# ---------------------- Paths, parámetros y switches ---------------------------
TX_PATH     = "../../data/tx/datos_trx__with_subsub_oficial.csv"
ALERTS_PATH = "../../data/alertas marzo-agosto 2025 enriched.csv"

SUBSEGMENT_REAL    = "Retail"     # para comparar con reales (customer_sub_type)
SUBSUBSEGMENTS_NEW = ["R-Low"]    # para escenarios (customer_sub_sub_type)

TYPE_FILTER = "Cash"              # pon None si NO quieres exigir Cash
ALERT_ON_CROSS_ONLY = True        # True => sólo cuando cruza el umbral

PARAMS = {
    "Actual": {"Amount": 299000000},
    "p95":    {"Amount": 16_861_936_597},
    "p97":    {"Amount": 17_640_863_445},
    "p99":    {"Amount": 18_419_790_292},
}

COUNT_FROM = pd.Timestamp("2025-02-21", tz="UTC")  # (se usa 30d previos para rolling)
COUNT_TO   = None  # ej: pd.Timestamp("2025-08-31 23:59:59", tz="UTC")

# ------------------------------- Helpers ---------------------------------------
def _to_dt_utc(s):
    return pd.to_datetime(s, errors="coerce", utc=True)

def _norm_txid_one(x):
    """'201657287.0' -> '201657287' ; NaN -> None ; strip espacios."""
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s.endswith(".0"):
        s = s[:-2]
    return s or None

def _norm_txid_list(cell) -> list[str]:
    """
    Para reales: '204060793, 204061006' -> ['204060793','204061006']
    Admite NaN. Normaliza '.0' si aparece.
    """
    if pd.isna(cell):
        return []
    raw = str(cell)
    parts = [p.strip() for p in raw.split(",") if p.strip() != ""]
    return [_norm_txid_one(p) for p in parts if p is not None]

def _load_tx(path):
    tx = pd.read_csv(path, dtype={"customer_id":"string","tx_id":"string"}, encoding="utf-8-sig", low_memory=False)
    tx["customer_sub_type"]      = tx.get("customer_sub_type","").astype(str)
    tx["customer_sub_sub_type"]  = tx.get("customer_sub_sub_type","").astype(str)
    tx["tx_direction"]   = tx.get("tx_direction","").astype(str).str.title()
    tx["tx_type"]        = tx.get("tx_type","").astype(str).str.title()
    tx["tx_base_amount"] = pd.to_numeric(tx.get("tx_base_amount"), errors="coerce")
    tx["tx_date_time"]   = _to_dt_utc(tx.get("tx_date_time"))
    # normaliza tx_id
    tx["tx_id_norm"] = tx["tx_id"].map(_norm_txid_one) if "tx_id" in tx.columns else pd.NA
    return tx

def _load_reales(path):
    dtypes = {
        "alert_id":"string",
        "rule_code":"string",
        "customer_id":"string",
        "tx_id":"string",
        "external_transaction_ids":"string",
        "number_of_transactions":"Int64",
        "customer_sub_type":"string",
    }
    real = pd.read_csv(path, dtype=dtypes, encoding="utf-8-sig", low_memory=False)
    real["rule_code"]        = real["rule_code"].astype(str).str.strip()
    real["customer_sub_type"]= real.get("customer_sub_type","").astype(str)
    real["tx_date_time"]     = _to_dt_utc(real.get("tx_date_time"))
    # extrae lista de ids (preferente) y también un set para match rápido
    real["ext_ids_list"] = real["external_transaction_ids"].map(_norm_txid_list)
    real["ext_ids_set"]  = real["ext_ids_list"].map(set)
    # backup: tx_id simple normalizado (por si algún registro no tiene lista)
    real["tx_id_norm"]   = real["tx_id"].map(_norm_txid_one)
    return real

def _simulate_triggers_inbound(df_in, amount, cross_only=True):
    """
    df_in: filas inbound (y tipo filtrado si aplica), con columnas:
           ['customer_id','tx_date_time','tx_base_amount','tx_id_norm']
    Devuelve df de gatillos por transacción con S30_before/S30_after.
    Nota clave: S30_before = S30_after - monto_actual  (para manejar varios en mismo timestamp).
    """
    if df_in.empty:
        cols = ["customer_id","tx_date_time","tx_base_amount","tx_id_norm","S30_before","S30_after"]
        return pd.DataFrame(columns=cols)

    parts = []
    for cid, sub in df_in.groupby("customer_id", sort=False):
        sub = sub.sort_values("tx_date_time").copy()
        amt_abs = sub["tx_base_amount"].abs().astype(float).values
        # Serie por transacción, indexada por datetime (puede tener duplicados)
        s = pd.Series(amt_abs, index=sub["tx_date_time"].values)
        S30_after = s.rolling("30D").sum().values  # alineado por posición
        S30_before = S30_after - amt_abs           # quitar sólo el monto actual

        sub["S30_after"]  = S30_after
        sub["S30_before"] = S30_before

        if cross_only:
            mask = (sub["S30_before"] <= amount) & (sub["S30_after"] > amount)
        else:
            mask = (sub["S30_after"] > amount)

        parts.append(sub.loc[mask, ["customer_id","tx_date_time","tx_base_amount","tx_id_norm","S30_before","S30_after"]])

    return pd.concat(parts, ignore_index=True) if parts else pd.DataFrame(
        columns=["customer_id","tx_date_time","tx_base_amount","tx_id_norm","S30_before","S30_after"]
    )

def _explain_window(tx_base, row, horizon_days=30):
    cid = row["customer_id"]
    t   = row["tx_date_time"]
    start = t - pd.Timedelta(days=horizon_days)
    sub = tx_base[(tx_base["customer_id"]==cid) & (tx_base["tx_date_time"]>start) & (tx_base["tx_date_time"]<=t)]
    return sub.sort_values("tx_date_time").copy()

# ------------------------------- Carga bases -----------------------------------
tx_all = _load_tx(TX_PATH)

# ===================== 1) Simulación "Actual" (segmento por customer_sub_type) =
tx_act = tx_all[tx_all["customer_sub_type"] == SUBSEGMENT_REAL].copy()
m = tx_act["tx_direction"].eq("Inbound") & tx_act["tx_base_amount"].notna() & tx_act["tx_date_time"].notna()
if TYPE_FILTER:
    m &= tx_act["tx_type"].eq(TYPE_FILTER)
g_act = tx_act.loc[m, ["customer_id","tx_date_time","tx_base_amount","tx_id_norm"]].copy()

A = float(PARAMS["Actual"]["Amount"])
sim_trig = _simulate_triggers_inbound(g_act, A, cross_only=ALERT_ON_CROSS_ONLY)
# cuenta/match solo desde COUNT_FROM (pero el rolling ya miró hacia atrás)
sim_trig = sim_trig[sim_trig["tx_date_time"] >= COUNT_FROM]
if COUNT_TO is not None:
    sim_trig = sim_trig[sim_trig["tx_date_time"] <= COUNT_TO]

# ===================== 2) Reales (P-HSUMI / SUBSEGMENT_REAL) ===================
real = _load_reales(ALERTS_PATH)
real = real[(real["rule_code"]=="P-HSUMI") & (real["customer_sub_type"]==SUBSEGMENT_REAL)].copy()
real = real[real["tx_date_time"].notna()]
real = real[(real["tx_date_time"] >= COUNT_FROM)]
if COUNT_TO is not None:
    real = real[(real["tx_date_time"] <= COUNT_TO)]

# ===================== 3) Matching (por cualquier tx_id de la alerta real) =====
sim_ids = set(sim_trig["tx_id_norm"].dropna().unique())
# crea una columna booleana: ¿algún id de la alerta real está en sim_ids?
real["matched_by_any_tx"] = real["ext_ids_set"].apply(lambda s: len(sim_ids.intersection(s)) > 0 if isinstance(s, set) else (pd.notna(real["tx_id_norm"]) and (real["tx_id_norm"] in sim_ids)))

reales_sin_sim = real[~real["matched_by_any_tx"]].copy()

# para sim_sin_reales, arma el set de todos los tx_ids referenciados por reales
real_all_ids = set().union(*real["ext_ids_set"].tolist()) if len(real) else set()
if "tx_id_norm" in real.columns:
    real_all_ids |= set(real["tx_id_norm"].dropna().unique().tolist())
sim_sin_reales = sim_trig[~sim_trig["tx_id_norm"].isin(real_all_ids)].copy()

# ===================== 4) Logs claros ==========================================
print("=== P-HSUMI — Resumen matching (Actual vs Reales) ===")
print(f"Alertas reales                     : {len(real):,}")
print(f"Alertas simuladas (gatillos)       : {len(sim_trig):,}")
print(f"Alertas reales SIN simulación      : {len(reales_sin_sim):,}")
print(f"Alertas en simulación SIN reales   : {len(sim_sin_reales):,}")

print("\n--- Reales sin simulación (head) ---")
display(reales_sin_sim[[
    "alert_id","rule_code","tx_date_time","number_of_transactions","external_transaction_ids"
]].head(15))

print("\n--- Simulación sin reales (head) ---")
display(sim_sin_reales[[
    "customer_id","tx_id_norm","tx_date_time","tx_base_amount","S30_before","S30_after"
]].head(15))

# ===================== 5) Explicación de casos simulados-sin-real ==============
N = 5
if not sim_sin_reales.empty:
    print(f"\n=== Ejemplos (primeros {N}) — por qué simuló y el real no ===")
    base_for_explain = tx_all[
        (tx_all["tx_direction"].eq("Inbound"))
        & (tx_all["tx_base_amount"].notna())
        & (tx_all["tx_date_time"].notna())
        & ((TYPE_FILTER is None) | tx_all["tx_type"].eq(TYPE_FILTER))
    ][["customer_id","tx_date_time","tx_base_amount","tx_id","tx_id_norm","tx_type"]].copy()

    for _, r in sim_sin_reales.head(N).iterrows():
        print("\n--- Caso:", dict(r[["customer_id","tx_id_norm","tx_date_time","S30_before","S30_after"]]))
        win = _explain_window(base_for_explain, r, horizon_days=30)
        win["amt_abs"] = win["tx_base_amount"].abs()
        print("Ventana (t-30d, t] que construye la suma (últimas 10 filas):")
        display(win.sort_values("tx_date_time").tail(10)[
            ["customer_id","tx_date_time","tx_id_norm","tx_type","tx_base_amount","amt_abs"]
        ])

# ===================== 6) Escenarios (subsub) por percentiles ===================
tx_new = tx_all[tx_all["customer_sub_sub_type"].isin(SUBSUBSEGMENTS_NEW)].copy()
m2 = tx_new["tx_direction"].eq("Inbound") & tx_new["tx_base_amount"].notna() & tx_new["tx_date_time"].notna()
if TYPE_FILTER:
    m2 &= tx_new["tx_type"].eq(TYPE_FILTER)
g_new = tx_new.loc[m2, ["customer_id","tx_date_time","tx_base_amount","tx_id_norm"]].copy()

scenario_counts = {}
for esc, pars in PARAMS.items():
    amt = float(pars["Amount"])
    trig = _simulate_triggers_inbound(g_new, amt, cross_only=ALERT_ON_CROSS_ONLY)
    trig = trig[trig["tx_date_time"] >= COUNT_FROM]
    if COUNT_TO is not None:
        trig = trig[trig["tx_date_time"] <= COUNT_TO]
    scenario_counts[esc] = int(trig.shape[0])

print("\n=== P-HSUMI — Escenarios (subsub) — #gatillos por escenario ===")
display(pd.DataFrame([scenario_counts]))

# === IDs de alertas reales que no están cubiertas por la simulación ===
ids_faltantes = (
    reales_sin_sim["alert_id"]
    .dropna()
    .astype("string")
    .unique()
    .tolist()
)

print(f"Total alertas reales sin simulación: {len(ids_faltantes):,}")
print("Primeros 50 alert_id faltantes:")
print(ids_faltantes[:50])

# (Opcional) ver detalles útiles para investigar:
cols_vista = [
    "alert_id",
    "tx_date_time",
    "number_of_transactions",
    "external_transaction_ids",
    "customer_id",
    "tx_direction",
    "tx_base_amount",
]
display(
    reales_sin_sim[cols_vista]
    .sort_values(["tx_date_time","alert_id"])
    .head(25)
)

# (Opcional) guardar todo para revisión
reales_sin_sim[cols_vista].to_csv("P-HSUMI_reales_sin_sim_detalle.csv", index=False, encoding="utf-8-sig")
print('Guardado: P-HSUMI_reales_sin_sim_detalle.csv')


=== P-HSUMI — Resumen matching (Actual vs Reales) ===
Alertas reales                     : 214
Alertas simuladas (gatillos)       : 124
Alertas reales SIN simulación      : 90
Alertas en simulación SIN reales   : 0

--- Reales sin simulación (head) ---


,alert_id,rule_code,tx_date_time,number_of_transactions,external_transaction_ids
33,80278,P-HSUMI,2025-02-21 00:00:00+00:00,1,201659826
575,81597,P-HSUMI,2025-02-25 00:00:00+00:00,1,201847693
1439,83614,P-HSUMI,2025-03-03 00:00:00+00:00,1,202250815
1624,83993,P-HSUMI,2025-03-04 00:00:00+00:00,1,202370432
1855,84878,P-HSUMI,2025-03-05 00:00:00+00:00,1,202454452
2168,85889,P-HSUMI,2025-03-06 00:00:00+00:00,1,202529237
2559,87312,P-HSUMI,2025-03-10 00:00:00+00:00,1,202726525
3030,88606,P-HSUMI,2025-03-12 00:00:00+00:00,1,202949753
3264,89705,P-HSUMI,2025-03-13 00:00:00+00:00,1,203080430
3688,91278,P-HSUMI,2025-03-17 00:00:00+00:00,1,203388498



--- Simulación sin reales (head) ---


,customer_id,tx_id_norm,tx_date_time,tx_base_amount,S30_before,S30_after


KeyboardInterrupt: 